In [1]:
import os
import gc
import scipy
import glob
import math
import torch
import pickle
import librosa
import warnings
import datetime
import numpy as np
import pandas as pd
import multiprocessing
from pathlib import Path
import matplotlib.pyplot as plt
import librosa.display
from sklearn.utils import shuffle
import random
#from feature_extractor import FeatureExtractor
#from utils import prepare_input_features
#from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import util
import soundfile as sf
import module_nn
from IPython.display import Audio 
warnings.filterwarnings(action='ignore')

In [2]:
def load_variables(path):
    with open(path, 'rb') as archivo:
        noise_files = pickle.load(archivo)
    return noise_files

In [3]:
noise_source_files = load_variables('variables/NOISES_STATIONARY_NOISE_S.pkl')
noise_target_files = load_variables('variables/NOISES_NO_STATIONARY_NOISE_T.pkl')
speakers_files = load_variables('variables/SPEAKERS_FOR_TESTING.pkl')

In [4]:
len(speakers_files[0])

100

In [5]:
speakers_audio = []
id_speaker = []
for i in range(len(speakers_files)):
    band = True
    for audio in speakers_files[i]:
        speakers_audio.append([i, audio])
        if(band):
            id_speaker.append([i, audio.split('/')[3]])
        band = False
speakers_audio

[[0, 'data/Speakers/train-clean-100/1723/1723-141149-0000.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0001.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0002.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0003.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0004.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0005.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0006.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0007.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0008.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0009.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0010.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0011.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0012.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0013.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0014.flac'],
 [0, 'data

In [6]:
id_speaker

[[0, '1723'],
 [1, '1737'],
 [2, '1743'],
 [3, '1841'],
 [4, '1867'],
 [5, '1898'],
 [6, '19'],
 [7, '1926'],
 [8, '196'],
 [9, '1963']]

In [7]:
#TAKE_SPEAKER_FROM_DNN = []
PATH_DNN_DENOISED = 'audios_denoised_dnn/'
AUDIOS_DNN = []
for paths_db in os.listdir(PATH_DNN_DENOISED):
    #print(paths_db)
    for paths_db_noise in os.listdir(os.path.join(PATH_DNN_DENOISED, paths_db)):
        for a in os.listdir(os.path.join(PATH_DNN_DENOISED, paths_db, paths_db_noise)):
            print(os.path.join(PATH_DNN_DENOISED, paths_db, paths_db_noise, a))
            AUDIOS_DNN.append(os.path.join(PATH_DNN_DENOISED, paths_db, paths_db_noise, a))
        #AUDIOS_DNN.append()

audios_denoised_dnn/9/stationary_typing/1898-145702-0002.flac
audios_denoised_dnn/9/stationary_typing/1926-147979-0030.flac
audios_denoised_dnn/9/stationary_typing/1926-147979-0013.flac
audios_denoised_dnn/9/stationary_typing/1867-148436-0035.flac
audios_denoised_dnn/9/stationary_typing/1723-141149-0078.flac
audios_denoised_dnn/9/stationary_typing/1841-150351-0015.flac
audios_denoised_dnn/9/stationary_typing/19-227-0040.flac
audios_denoised_dnn/9/stationary_typing/1898-145702-0001.flac
audios_denoised_dnn/9/stationary_typing/1963-142393-0002.flac
audios_denoised_dnn/9/stationary_typing/1841-179183-0011.flac
audios_denoised_dnn/9/stationary_typing/19-198-0037.flac
audios_denoised_dnn/9/stationary_typing/1867-154071-0031.flac
audios_denoised_dnn/9/stationary_typing/19-227-0030.flac
audios_denoised_dnn/9/stationary_typing/1898-145724-0005.flac
audios_denoised_dnn/9/stationary_typing/1723-141149-0052.flac
audios_denoised_dnn/9/stationary_typing/19-227-0001.flac
audios_denoised_dnn/9/statio

In [9]:
random.seed(999)
AUDIOS_DNN_SELECTED = random.sample(AUDIOS_DNN, 1000)
util.generate_pkl('variables/', AUDIOS_DNN_SELECTED, 'AUDIOS_DNN_SELECTED')
AUDIOS_DNN_SELECTED

['audios_denoised_dnn/9/stationary_rain/1867-154071-0031.flac', 'audios_denoised_dnn/6/stationary_car/1926-147979-0013.flac', 'audios_denoised_dnn/6/stationary_cabin/1898-145702-0002.flac', 'audios_denoised_dnn/6/stationary_pink/1867-154071-0005.flac', 'audios_denoised_dnn/-3/stationary_cabin/1867-148436-0012.flac', 'audios_denoised_dnn/-3/stationary_car/1841-150351-0032.flac', 'audios_denoised_dnn/3/nonstationary_babycry/1963-142393-0013.flac', 'audios_denoised_dnn/0/stationary_cabin/19-227-0030.flac', 'audios_denoised_dnn/3/nonstationary_dog_bark/1743-142912-0029.flac', 'audios_denoised_dnn/-6/nonstationary_crowd_party_adult_med/19-198-0003.flac', 'audios_denoised_dnn/3/stationary_cabin/1926-143879-0007.flac', 'audios_denoised_dnn/0/nonstationary_dog_bark/1723-141149-0011.flac', 'audios_denoised_dnn/6/stationary_car/1898-145720-0003.flac', 'audios_denoised_dnn/9/stationary_wind/1898-145720-0009.flac', 'audios_denoised_dnn/-6/stationary_rain/196-122152-0021.flac', 'audios_denoised_dnn

['audios_denoised_dnn/9/stationary_rain/1867-154071-0031.flac',
 'audios_denoised_dnn/6/stationary_car/1926-147979-0013.flac',
 'audios_denoised_dnn/6/stationary_cabin/1898-145702-0002.flac',
 'audios_denoised_dnn/6/stationary_pink/1867-154071-0005.flac',
 'audios_denoised_dnn/-3/stationary_cabin/1867-148436-0012.flac',
 'audios_denoised_dnn/-3/stationary_car/1841-150351-0032.flac',
 'audios_denoised_dnn/3/nonstationary_babycry/1963-142393-0013.flac',
 'audios_denoised_dnn/0/stationary_cabin/19-227-0030.flac',
 'audios_denoised_dnn/3/nonstationary_dog_bark/1743-142912-0029.flac',
 'audios_denoised_dnn/-6/nonstationary_crowd_party_adult_med/19-198-0003.flac',
 'audios_denoised_dnn/3/stationary_cabin/1926-143879-0007.flac',
 'audios_denoised_dnn/0/nonstationary_dog_bark/1723-141149-0011.flac',
 'audios_denoised_dnn/6/stationary_car/1898-145720-0003.flac',
 'audios_denoised_dnn/9/stationary_wind/1898-145720-0009.flac',
 'audios_denoised_dnn/-6/stationary_rain/196-122152-0021.flac',
 'audi

In [10]:
AUDIO_DNN_ID = []
for audio_dnn_file in AUDIOS_DNN_SELECTED:
    print(audio_dnn_file.split('/')[3].split('-')[0])
    print(audio_dnn_file)
    for id_ in id_speaker:
        if(audio_dnn_file.split('/')[3].split('-')[0]==id_[1]):
            AUDIO_DNN_ID.append([id_[0], audio_dnn_file])
            print(id_)
            print('id_finded')

1867
audios_denoised_dnn/9/stationary_rain/1867-154071-0031.flac
[4, '1867']
id_finded
1926
audios_denoised_dnn/6/stationary_car/1926-147979-0013.flac
[7, '1926']
id_finded
1898
audios_denoised_dnn/6/stationary_cabin/1898-145702-0002.flac
[5, '1898']
id_finded
1867
audios_denoised_dnn/6/stationary_pink/1867-154071-0005.flac
[4, '1867']
id_finded
1867
audios_denoised_dnn/-3/stationary_cabin/1867-148436-0012.flac
[4, '1867']
id_finded
1841
audios_denoised_dnn/-3/stationary_car/1841-150351-0032.flac
[3, '1841']
id_finded
1963
audios_denoised_dnn/3/nonstationary_babycry/1963-142393-0013.flac
[9, '1963']
id_finded
19
audios_denoised_dnn/0/stationary_cabin/19-227-0030.flac
[6, '19']
id_finded
1743
audios_denoised_dnn/3/nonstationary_dog_bark/1743-142912-0029.flac
[2, '1743']
id_finded
19
audios_denoised_dnn/-6/nonstationary_crowd_party_adult_med/19-198-0003.flac
[6, '19']
id_finded
1926
audios_denoised_dnn/3/stationary_cabin/1926-143879-0007.flac
[7, '1926']
id_finded
1723
audios_denoised_dn

In [11]:
AUDIO_DNN_ID

[[4, 'audios_denoised_dnn/9/stationary_rain/1867-154071-0031.flac'],
 [7, 'audios_denoised_dnn/6/stationary_car/1926-147979-0013.flac'],
 [5, 'audios_denoised_dnn/6/stationary_cabin/1898-145702-0002.flac'],
 [4, 'audios_denoised_dnn/6/stationary_pink/1867-154071-0005.flac'],
 [4, 'audios_denoised_dnn/-3/stationary_cabin/1867-148436-0012.flac'],
 [3, 'audios_denoised_dnn/-3/stationary_car/1841-150351-0032.flac'],
 [9, 'audios_denoised_dnn/3/nonstationary_babycry/1963-142393-0013.flac'],
 [6, 'audios_denoised_dnn/0/stationary_cabin/19-227-0030.flac'],
 [2, 'audios_denoised_dnn/3/nonstationary_dog_bark/1743-142912-0029.flac'],
 [6,
  'audios_denoised_dnn/-6/nonstationary_crowd_party_adult_med/19-198-0003.flac'],
 [7, 'audios_denoised_dnn/3/stationary_cabin/1926-143879-0007.flac'],
 [0, 'audios_denoised_dnn/0/nonstationary_dog_bark/1723-141149-0011.flac'],
 [5, 'audios_denoised_dnn/6/stationary_car/1898-145720-0003.flac'],
 [5, 'audios_denoised_dnn/9/stationary_wind/1898-145720-0009.flac']

In [12]:
random.seed(999)
np.random.shuffle(speakers_audio)
speakers_audio

[[3, 'data/Speakers/train-clean-100/1841/1841-159771-0006.flac'],
 [4, 'data/Speakers/train-clean-100/1867/1867-148436-0008.flac'],
 [9, 'data/Speakers/train-clean-100/1963/1963-142393-0049.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0054.flac'],
 [3, 'data/Speakers/train-clean-100/1841/1841-150351-0027.flac'],
 [0, 'data/Speakers/train-clean-100/1723/1723-141149-0012.flac'],
 [9, 'data/Speakers/train-clean-100/1963/1963-147036-0000.flac'],
 [1, 'data/Speakers/train-clean-100/1737/1737-142397-0009.flac'],
 [6, 'data/Speakers/train-clean-100/19/19-198-0002.flac'],
 [7, 'data/Speakers/train-clean-100/1926/1926-147979-0027.flac'],
 [8, 'data/Speakers/train-clean-100/196/196-122159-0021.flac'],
 [1, 'data/Speakers/train-clean-100/1737/1737-146161-0002.flac'],
 [8, 'data/Speakers/train-clean-100/196/196-122152-0014.flac'],
 [2, 'data/Speakers/train-clean-100/1743/1743-142913-0014.flac'],
 [1, 'data/Speakers/train-clean-100/1737/1737-146161-0003.flac'],
 [4, 'data/Speakers/t

In [13]:
#get sets of data
def get_list_of_sets(data_list):
    # lista con las 1500 muestras
    data = data_list
    # número de muestras para cada conjunto
    train_size = int(0.8 * len(data))
    val_size = int(0.1 * len(data))
    test_size = len(data) - train_size - val_size
    # separar las muestras en conjuntos
    train_data = data[:train_size]
    val_data = data[train_size:train_size+val_size]
    test_data = data[train_size+val_size:]
    print(f"Tamaño del conjunto de entrenamiento: {len(train_data)}")
    print(f"Tamaño del conjunto de validación: {len(val_data)}")
    print(f"Tamaño del conjunto de prueba: {len(test_data)}")
    return train_data, val_data, test_data

In [14]:
train_speaker_clean, val_speaker_clean, test_speaker_clean = get_list_of_sets(speakers_audio)
train_speaker_dnn, val_speaker_dnn, test_speaker_dnn = get_list_of_sets(AUDIO_DNN_ID)

Tamaño del conjunto de entrenamiento: 800
Tamaño del conjunto de validación: 100
Tamaño del conjunto de prueba: 100
Tamaño del conjunto de entrenamiento: 800
Tamaño del conjunto de validación: 100
Tamaño del conjunto de prueba: 100


In [15]:
util.generate_pkl('variables/', train_speaker_clean, 'train_speaker_identification_clean')
util.generate_pkl('variables/', val_speaker_clean, 'val_speaker_identification_clean')
util.generate_pkl('variables/', test_speaker_clean, 'test_speaker_identification_clean')

util.generate_pkl('variables/', train_speaker_dnn, 'train_speaker_identification_clean_dnn')
util.generate_pkl('variables/', val_speaker_dnn, 'val_speaker_identification_clean_dnn')
util.generate_pkl('variables/', test_speaker_dnn, 'test_speaker_identification_clean_dnn')

[[3, 'data/Speakers/train-clean-100/1841/1841-159771-0006.flac'], [4, 'data/Speakers/train-clean-100/1867/1867-148436-0008.flac'], [9, 'data/Speakers/train-clean-100/1963/1963-142393-0049.flac'], [0, 'data/Speakers/train-clean-100/1723/1723-141149-0054.flac'], [3, 'data/Speakers/train-clean-100/1841/1841-150351-0027.flac'], [0, 'data/Speakers/train-clean-100/1723/1723-141149-0012.flac'], [9, 'data/Speakers/train-clean-100/1963/1963-147036-0000.flac'], [1, 'data/Speakers/train-clean-100/1737/1737-142397-0009.flac'], [6, 'data/Speakers/train-clean-100/19/19-198-0002.flac'], [7, 'data/Speakers/train-clean-100/1926/1926-147979-0027.flac'], [8, 'data/Speakers/train-clean-100/196/196-122159-0021.flac'], [1, 'data/Speakers/train-clean-100/1737/1737-146161-0002.flac'], [8, 'data/Speakers/train-clean-100/196/196-122152-0014.flac'], [2, 'data/Speakers/train-clean-100/1743/1743-142913-0014.flac'], [1, 'data/Speakers/train-clean-100/1737/1737-146161-0003.flac'], [4, 'data/Speakers/train-clean-100/

In [16]:
train_speaker_dnn

[[4, 'audios_denoised_dnn/9/stationary_rain/1867-154071-0031.flac'],
 [7, 'audios_denoised_dnn/6/stationary_car/1926-147979-0013.flac'],
 [5, 'audios_denoised_dnn/6/stationary_cabin/1898-145702-0002.flac'],
 [4, 'audios_denoised_dnn/6/stationary_pink/1867-154071-0005.flac'],
 [4, 'audios_denoised_dnn/-3/stationary_cabin/1867-148436-0012.flac'],
 [3, 'audios_denoised_dnn/-3/stationary_car/1841-150351-0032.flac'],
 [9, 'audios_denoised_dnn/3/nonstationary_babycry/1963-142393-0013.flac'],
 [6, 'audios_denoised_dnn/0/stationary_cabin/19-227-0030.flac'],
 [2, 'audios_denoised_dnn/3/nonstationary_dog_bark/1743-142912-0029.flac'],
 [6,
  'audios_denoised_dnn/-6/nonstationary_crowd_party_adult_med/19-198-0003.flac'],
 [7, 'audios_denoised_dnn/3/stationary_cabin/1926-143879-0007.flac'],
 [0, 'audios_denoised_dnn/0/nonstationary_dog_bark/1723-141149-0011.flac'],
 [5, 'audios_denoised_dnn/6/stationary_car/1898-145720-0003.flac'],
 [5, 'audios_denoised_dnn/9/stationary_wind/1898-145720-0009.flac']

In [17]:
NOISES = noise_source_files + noise_target_files
SNR = [-12, -6, -3]

In [18]:
#NOISES.pop(1)

In [19]:
#NOISES.pop(2)

In [20]:
NOISES

['data/NOISE/nonstationary/babycry.wav',
 'data/NOISE/nonstationary/crowd_party_adult_med.wav',
 'data/NOISE/nonstationary/bell_church.wav',
 'data/NOISE/nonstationary/cafeteria_babble.wav',
 'data/NOISE/nonstationary/helicopter.wav',
 'data/NOISE/nonstationary/people.wav',
 'data/NOISE/nonstationary/dog_bark.wav',
 'data/NOISE/stationary/pink.wav',
 'data/NOISE/stationary/dripping_water.wav',
 'data/NOISE/stationary/car.wav',
 'data/NOISE/stationary/cabin.wav',
 'data/NOISE/stationary/rain.wav',
 'data/NOISE/stationary/wind.wav',
 'data/NOISE/stationary/typing.wav']

In [21]:
def generate_data_audio_noise(path_speaker_noise, list_audios, noises, snr):
    #print(path_speaker_noise)
    PATH_AUDIO_NOISES = []
    for data_set in list_audios:
        AUX_PATH_NOISE = []
        for id_audio in data_set:
            int_snr = random.randint(0, len(snr)-1)
            int_noises = random.randint(0, len(noises)-1)
            #print(id_audio[1].split('/'))
            aux_list_path = id_audio[1].split('/')
            aux_list_path[1] = path_speaker_noise.split('/')[1]
            #print("/".join(aux_list_path[0:len(aux_list_path)-1]))
            Path("/".join(aux_list_path[0:len(aux_list_path)-1])).mkdir(parents=True, exist_ok=True)
            #print("/".join(aux_list_path[0:len(aux_list_path)]))
            audio_clean, _ = librosa.load(id_audio[1], sr=16000)
            audio_noise, _ = librosa.load(noises[int_noises])
            mixed_audio = util.add_noise_to_clean_audio(audio_clean, audio_noise, snr[int_snr])
            sf.write("/".join(aux_list_path[0:len(aux_list_path)]), mixed_audio, samplerate=16000, subtype='PCM_16')
            AUX_PATH_NOISE.append([id_audio[0], "/".join(aux_list_path[0:len(aux_list_path)]), noises[int_noises], snr[int_snr]])
        PATH_AUDIO_NOISES.append(AUX_PATH_NOISE)
            #print(len(audio_clean))
            #print(len(audio_noise))
    return PATH_AUDIO_NOISES

In [22]:
path_noise_speaker_clean = 'data/Speakers_with_noise_clean/'
PATH_AUDIO_NOISES_CLEAN = generate_data_audio_noise(path_noise_speaker_clean, [train_speaker_clean, val_speaker_clean, test_speaker_clean], NOISES, SNR)

In [23]:
train_speaker_noises_clean = PATH_AUDIO_NOISES_CLEAN[0]
val_speaker_noises_clean = PATH_AUDIO_NOISES_CLEAN[1]
test_speaker_noises_clean = PATH_AUDIO_NOISES_CLEAN[2]

In [24]:
util.generate_pkl('variables/', train_speaker_noises_clean, 'train_speaker_noises_identification_noise_clean')
util.generate_pkl('variables/', val_speaker_noises_clean, 'val_speaker_noises_identification_noise_clean')
util.generate_pkl('variables/', test_speaker_noises_clean, 'test_speaker_noises_identification_noise_clean')

[[3, 'data/Speakers_with_noise_clean/train-clean-100/1841/1841-159771-0006.flac', 'data/NOISE/nonstationary/crowd_party_adult_med.wav', -3], [4, 'data/Speakers_with_noise_clean/train-clean-100/1867/1867-148436-0008.flac', 'data/NOISE/stationary/car.wav', -3], [9, 'data/Speakers_with_noise_clean/train-clean-100/1963/1963-142393-0049.flac', 'data/NOISE/stationary/pink.wav', -3], [0, 'data/Speakers_with_noise_clean/train-clean-100/1723/1723-141149-0054.flac', 'data/NOISE/nonstationary/bell_church.wav', -6], [3, 'data/Speakers_with_noise_clean/train-clean-100/1841/1841-150351-0027.flac', 'data/NOISE/nonstationary/people.wav', -3], [0, 'data/Speakers_with_noise_clean/train-clean-100/1723/1723-141149-0012.flac', 'data/NOISE/stationary/wind.wav', -3], [9, 'data/Speakers_with_noise_clean/train-clean-100/1963/1963-147036-0000.flac', 'data/NOISE/stationary/typing.wav', -3], [1, 'data/Speakers_with_noise_clean/train-clean-100/1737/1737-142397-0009.flac', 'data/NOISE/stationary/cabin.wav', -12], [

In [25]:
train_speaker_noises_clean = util.load_variables('variables/train_speaker_noises_identification_noise_clean.pkl')
val_speaker_noises_clean = util.load_variables('variables/val_speaker_noises_identification_noise_clean.pkl')
test_speaker_noises_clean = util.load_variables('variables/test_speaker_noises_identification_noise_clean.pkl')

In [26]:
path_noise_speaker_dnn = 'data/Speakers_with_noise_dnn/'
PATH_AUDIO_NOISES_DNN = generate_data_audio_noise(path_noise_speaker_dnn, [train_speaker_dnn, val_speaker_dnn, test_speaker_dnn], NOISES, SNR)

In [27]:
train_speaker_noises_dnn = PATH_AUDIO_NOISES_DNN[0]
val_speaker_noises_dnn = PATH_AUDIO_NOISES_DNN[1]
test_speaker_noises_dnn = PATH_AUDIO_NOISES_DNN[2]

In [28]:
util.generate_pkl('variables/', train_speaker_noises_dnn, 'train_speaker_noises_identification_noise_dnn')
util.generate_pkl('variables/', val_speaker_noises_dnn, 'val_speaker_noises_identification_noise_dnn')
util.generate_pkl('variables/', test_speaker_noises_dnn, 'test_speaker_noises_identification_noise_dnn')

[[4, 'audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain/1867-154071-0031.flac', 'data/NOISE/nonstationary/crowd_party_adult_med.wav', -12], [7, 'audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/1926-147979-0013.flac', 'data/NOISE/nonstationary/dog_bark.wav', -6], [5, 'audios_denoised_dnn/Speakers_with_noise_dnn/stationary_cabin/1898-145702-0002.flac', 'data/NOISE/stationary/dripping_water.wav', -3], [4, 'audios_denoised_dnn/Speakers_with_noise_dnn/stationary_pink/1867-154071-0005.flac', 'data/NOISE/nonstationary/babycry.wav', -3], [4, 'audios_denoised_dnn/Speakers_with_noise_dnn/stationary_cabin/1867-148436-0012.flac', 'data/NOISE/nonstationary/helicopter.wav', -3], [3, 'audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/1841-150351-0032.flac', 'data/NOISE/nonstationary/crowd_party_adult_med.wav', -6], [9, 'audios_denoised_dnn/Speakers_with_noise_dnn/nonstationary_babycry/1963-142393-0013.flac', 'data/NOISE/stationary/wind.wav', -12], [6, 'audios_denoised_dn